In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os

path = "C:\\Users\\Playdata\\OneDrive\\Desktop\\SKN09-3nd-4Team\\data"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

file_list: ['00120250228.pdf', '00220250228.pdf', '00320250228.pdf', '00420250228.pdf', '00520250228.pdf', '00620250228.pdf', '00720250228.pdf', '00820250228.pdf', '00920250228.pdf', '01020250228.pdf', '01120250228.pdf', '01220250228.pdf', '01320250201.pdf', '01420250201.pdf', '01520250101.pdf', '01620250101.pdf', '01720250201.pdf', '01820250201.pdf', '01920250101.pdf', '02020250201.pdf', '02120250101.pdf', '02220220101.pdf', '02320250201.pdf', '02420250101.pdf', '02520111001.pdf', '02620250101.pdf', '02720250101.pdf', '02820250101.pdf', '02920250101.pdf', '03020250101.pdf', '03120250101.pdf', '03220250101.pdf', '03320250101.pdf', '03420250101.pdf', '03520250101.pdf', '03620250101.pdf', '03720250101.pdf', '03820250101.pdf', '03920250101.pdf']


In [ ]:
!pip install transformers

----

# ERROR

In [ ]:
import faiss
import pickle
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer

model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # pool_mask 생성 (입력 시퀀스 길이에 맞춰서 1로 설정)
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)  # pool_mask 추가

    return outputs.embedding.numpy()

# 1️⃣ FAISS 저장 경로
faiss_index_path = "./faiss_index_1.bin"
metadata_path = "./documents_1.pkl"

# 2️⃣ 임베딩 모델 로드 (Sentence-BERT 사용)
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")



## 변환
def get_embeddings(texts):
    """텍스트 리스트를 입력받아 임베딩을 반환하는 함수"""
    return embedding_model.encode(texts, convert_to_numpy=True)

# 3️⃣ PDF 문서 로드
documents = []  # 모든 문서를 저장할 리스트
for file in file_list[:39]:  # 로드 할 파일 갯수
    loader = PyPDFLoader(path + "\\" + file)
    documents.extend(loader.load())  # ✅ 여러 문서를 리스트에 추가

# 4️⃣ 문서 전처리
for doc in documents:
    doc.page_content = doc.page_content.replace("\n", " ").replace("  ", " ")
    doc.metadata["source"] = file  # ✅ 문서의 출처 정보를 추가

# 5️⃣ 문서 스플리팅
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
split_documents = splitter.split_documents(documents)  # ✅ 리스트 전체를 입력해야 함

# 6️⃣ 스플릿된 문서로 벡터 생성
batch_size = 16
texts = [doc.page_content for doc in split_documents]
embeddings = []

for i in range(0, len(texts), batch_size):
    batch = texts[i : i + batch_size]
    batch_embeddings = get_embeddings(batch)  # 🔥 배치 단위로 임베딩 생성
    embeddings.extend(batch_embeddings)

# 7️⃣ FAISS 인덱스 생성 및 저장
embedding_dim = len(embeddings[0])
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings, dtype=np.float32))

faiss.write_index(index, faiss_index_path)
with open(metadata_path, "wb") as f:
    pickle.dump(split_documents, f)

print("✅ FAISS 인덱스 및 문서 저장 완료!")
print(f"FAISS에 저장된 벡터 개수: {index.ntotal}")
print(f"저장된 문서 개수: {len(documents)}")

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [4]:
import faiss
import pickle
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer

model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # pool_mask 생성 (입력 시퀀스 길이에 맞춰서 1로 설정)
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)  # pool_mask 추가

    return outputs.embedding.numpy()

# 1️⃣ FAISS 저장 경로
faiss_index_path = "./faiss_index_1.bin"
metadata_path = "./documents_1.pkl"

# 2️⃣ 임베딩 모델 로드 (Sentence-BERT 사용)
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")



## 변환
def get_embeddings(texts):
    """텍스트 리스트를 입력받아 임베딩을 반환하는 함수"""
    return embedding_model.encode(texts, convert_to_numpy=True)

# 3️⃣ PDF 문서 로드
documents = []  # 모든 문서를 저장할 리스트
for file in file_list[:39]:  # 로드 할 파일 갯수
    loader = PyPDFLoader(path + "\\" + file)
    documents.extend(loader.load())  # ✅ 여러 문서를 리스트에 추가

# 4️⃣ 문서 전처리
for doc in documents:
    doc.page_content = doc.page_content.replace("\n", " ").replace("  ", " ")
    doc.metadata["source"] = file  # ✅ 문서의 출처 정보를 추가

# 5️⃣ 문서 스플리팅
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
split_documents = splitter.split_documents(documents)  # ✅ 리스트 전체를 입력해야 함

# 6️⃣ 스플릿된 문서로 벡터 생성
batch_size = 16
texts = [doc.page_content for doc in split_documents]
embeddings = []

for i in range(0, len(texts), batch_size):
    batch = texts[i : i + batch_size]
    batch_embeddings = get_embeddings(batch)  # 🔥 배치 단위로 임베딩 생성
    embeddings.extend(batch_embeddings)

# 7️⃣ FAISS 인덱스 생성 및 저장
embedding_dim = len(embeddings[0])
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings, dtype=np.float32))

faiss.write_index(index, faiss_index_path)
with open(metadata_path, "wb") as f:
    pickle.dump(split_documents, f)

print("✅ FAISS 인덱스 및 문서 저장 완료!")
print(f"FAISS에 저장된 벡터 개수: {index.ntotal}")
print(f"저장된 문서 개수: {len(documents)}")

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [6]:
import os
import faiss
import pickle
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

# 1️⃣ 경로 설정
pdf_folder = "./axa_pdfs"
faiss_index_path = "./faiss_index_1.bin"
metadata_path = "./documents_1.pkl"

# 2️⃣ 임베딩 모델 로드
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

def get_embeddings(texts):
    """텍스트 리스트를 임베딩 벡터로 변환"""
    if not texts or not isinstance(texts, list):
        raise ValueError("입력은 문자열 리스트여야 합니다.")
    texts = [str(t).strip() for t in texts if t and isinstance(t, str)]
    return embedding_model.encode(texts, convert_to_numpy=True)

# 3️⃣ PDF 문서 로드 및 전처리
documents = []
file_list = sorted(os.listdir(pdf_folder))[:39]

for file in file_list:
    loader = PyPDFLoader(os.path.join(pdf_folder, file))
    pdf_docs = loader.load()
    for doc in pdf_docs:
        doc.page_content = doc.page_content.replace("\n", " ").replace("  ", " ")
        doc.metadata["source"] = file
    documents.extend(pdf_docs)

# 4️⃣ 문서 분할
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
split_documents = splitter.split_documents(documents)

# 5️⃣ 임베딩 생성
batch_size = 16
texts = [doc.page_content for doc in split_documents]
embeddings = []

for i in range(0, len(texts), batch_size):
    batch = texts[i : i + batch_size]
    batch_embeddings = get_embeddings(batch)
    embeddings.extend(batch_embeddings)

# 6️⃣ FAISS 인덱스 생성 및 저장
embedding_dim = len(embeddings[0])
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings, dtype=np.float32))

faiss.write_index(index, faiss_index_path)

# 7️⃣ 문서 메타데이터 저장
with open(metadata_path, "wb") as f:
    pickle.dump(split_documents, f)

print("✅ FAISS 인덱스 및 문서 저장 완료!")
print(f"FAISS에 저장된 벡터 수: {index.ntotal}")
print(f"총 문서 수: {len(documents)} | 분할 문서 수: {len(split_documents)}")


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [7]:
import os
import faiss
import pickle
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

# 1️⃣ 설정
pdf_path = "./axa_pdfs"
file_list = sorted(os.listdir(pdf_path))[:39]  # 최대 39개 파일 사용
faiss_index_path = "./faiss_index_1.bin"
metadata_path = "./documents_1.pkl"

# 2️⃣ 임베딩 모델 로드
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 3️⃣ 안전한 임베딩 함수
def get_embeddings(texts):
    if not texts or not isinstance(texts, list):
        raise ValueError("입력은 문자열 리스트여야 합니다.")
    
    cleaned = []
    for t in texts:
        if isinstance(t, str):
            stripped = t.strip()
            if stripped:
                cleaned.append(stripped)

    if not cleaned:
        raise ValueError("입력된 리스트에 유효한 문자열이 없습니다.")
    
    return embedding_model.encode(cleaned, convert_to_numpy=True)

# 4️⃣ PDF 문서 로드 및 전처리
documents = []
for file in file_list:
    loader = PyPDFLoader(os.path.join(pdf_path, file))
    loaded_docs = loader.load()
    for doc in loaded_docs:
        doc.page_content = str(doc.page_content).replace("\n", " ").replace("  ", " ").strip()
        doc.metadata["source"] = file
        documents.append(doc)

# 5️⃣ 문서 스플리팅
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
split_documents = splitter.split_documents(documents)

# 6️⃣ 텍스트 임베딩 (배치처리)
texts = [str(doc.page_content).strip() for doc in split_documents if doc.page_content]
batch_size = 16
embeddings = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    batch_embeddings = get_embeddings(batch)
    embeddings.extend(batch_embeddings)

# 7️⃣ FAISS 인덱스 생성 및 저장
embedding_dim = len(embeddings[0])
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings, dtype=np.float32))

faiss.write_index(index, faiss_index_path)
with open(metadata_path, "wb") as f:
    pickle.dump(split_documents, f)

print("✅ FAISS 인덱스 및 문서 저장 완료!")
print(f"FAISS 벡터 수: {index.ntotal}")
print(f"전체 문서 수: {len(documents)} | 분할된 문서 수: {len(split_documents)}")


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
# 1️⃣ 저장된 FAISS 인덱스 로드
index = faiss.read_index("./faiss_index_1.bin")

# 2️⃣ 문서 정보 로드
with open("./documents_1.pkl", "ab") as f:
    documents = pickle.load(f)

# 3️⃣ 검색 수행
query = " 보험금의 종류 및 한도에 대해 설명해줘?"
# query = "의무보험 미가입에 따른 불이익을 알려줘"
query_embedding = get_embeddings([query])[0]  # 쿼리 임베딩
query_embedding = np.array([query_embedding], dtype=np.float32)

D, I = index.search(query_embedding, k=5)  # 가장 유사한 5개 검색

# 4️⃣ 검색 결과 출력
for idx in I[0]:
    print(f"🔹 문서 {idx}: {documents[idx].page_content[:200]}")


----

# 정상 작동

- 문제 원인	해결 방법
- None 값 포함	if doc.page_content: 로 체크
- page_content에 공백만 있는 경우	.strip() 사용
- int, float 등 str 아닌 경우	str(doc.page_content) 강제 변환
- 배치별 오류 방지	try-except로 각 배치 임베딩 보호
- 문제가 있는 PDF 경고 메시지 출력
- print(f"⚠️ {file} 에서 유효한 텍스트를 찾을 수 없음. 스킵합니다.")
- 특정 PDF에서 텍스트를 추출할 수 없는 경우 사용자에게 알림.

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import os

path = "C:\\Users\\Playdata\\OneDrive\\Desktop\\SKN09-3nd-4Team\\data"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

file_list: ['00120250228.pdf', '00220250228.pdf', '00320250228.pdf', '00420250228.pdf', '00520250228.pdf', '00620250228.pdf', '00720250228.pdf', '00820250228.pdf', '00920250228.pdf', '01020250228.pdf', '01120250228.pdf', '01220250228.pdf', '01320250201.pdf', '01420250201.pdf', '01520250101.pdf', '01620250101.pdf', '01720250201.pdf', '01820250201.pdf', '01920250101.pdf', '02020250201.pdf', '02120250101.pdf', '02220220101.pdf', '02320250201.pdf', '02420250101.pdf', '02520111001.pdf', '02620250101.pdf', '02720250101.pdf', '02820250101.pdf', '02920250101.pdf', '03020250101.pdf', '03120250101.pdf', '03220250101.pdf', '03320250101.pdf', '03420250101.pdf', '03520250101.pdf', '03620250101.pdf', '03720250101.pdf', '03820250101.pdf', '03920250101.pdf']


In [10]:
import os
import faiss
import pickle
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

# 설정
pdf_path = "./data"
file_list = sorted(os.listdir(pdf_path))
faiss_index_path = "./faiss_index_1.bin"
metadata_path = "./documents_1.pkl"

# 모델 로드
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")   # OK
# embedding_model = "Bllossom/llama-3.2-Korean-Bllossom-3B"    # 
# embedding_model = "kakaocorp/kanana-nano-2.1b-embedding"   # 

# 안전한 임베딩 함수
def get_embeddings(texts):
    if not isinstance(texts, list):
        raise ValueError("입력은 문자열 리스트여야 합니다.")

    # str이 아니거나 빈 문자열 제거
    cleaned = [str(t).strip() for t in texts if isinstance(t, str) and t.strip()]
    if not cleaned:
        raise ValueError("입력된 리스트에 유효한 텍스트가 없습니다.")

    return embedding_model.encode(cleaned, convert_to_numpy=True)

# 문서 로드 및 전처리
documents = []
for file in file_list[:39]:
    loader = PyPDFLoader(os.path.join(pdf_path, file))
    docs = loader.load()
    for doc in docs:
        if doc.page_content:
            text = str(doc.page_content).replace("\n", " ").replace("  ", " ").strip()
            if text:  # 비어 있지 않으면 추가
                doc.page_content = text
                doc.metadata["source"] = file
                documents.append(doc)

# 문서 분할
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
split_documents = splitter.split_documents(documents)

# 텍스트 추출
texts = [doc.page_content for doc in split_documents if isinstance(doc.page_content, str) and doc.page_content.strip()]
batch_size = 16
embeddings = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    try:
        batch_embeddings = get_embeddings(batch)
        embeddings.extend(batch_embeddings)
    except Exception as e:
        print(f" 임베딩 실패 (배치 {i}-{i+batch_size}): {e}")
        continue

# FAISS 인덱스 생성
embedding_dim = len(embeddings[0])
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings, dtype=np.float32))

# 저장
faiss.write_index(index, faiss_index_path)
with open(metadata_path, "wb") as f:
    pickle.dump(split_documents, f)

print("FAISS 인덱스 및 문서 저장 완료")
print(f" 총 벡터 수: {index.ntotal}")
print(f" 원본 문서 수: {len(documents)} | 분할 문서 수: {len(split_documents)}")

 임베딩 실패 (배치 17904-17920): TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
FAISS 인덱스 및 문서 저장 완료
 총 벡터 수: 27457
 원본 문서 수: 8239 | 분할 문서 수: 27473


In [ ]:
# 임베딩 실패 시 해당 문서 저장
failed_batches = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    try:
        batch_embeddings = get_embeddings(batch)
        embeddings.extend(batch_embeddings)
    except Exception as e:
        failed_batches.append((i, i+batch_size, e))


In [11]:
import faiss
import pickle
import numpy as np

# 저장된 FAISS 인덱스 로드
index = faiss.read_index("./faiss_index_1.bin")

# 문서 정보 로드  (읽기 모드로 수정!)
with open("./documents_1.pkl", "rb") as f:
    documents = pickle.load(f)

# 검색 수행
query = "자동차 대인 대물보험에 대해?"
query_embedding = get_embeddings([query])[0]  # 쿼리 임베딩
query_embedding = np.array([query_embedding], dtype=np.float32)

D, I = index.search(query_embedding, k=5)  # 가장 유사한 5개 검색

# 검색 결과 출력
print("검색 결과:")
for idx in I[0]:
    if idx < len(documents):  # IndexError 방지
        print(f"🔹 문서 {idx}: {documents[idx].page_content[:200]}")
    else:
        print(f"⚠️ 잘못된 인덱스: {idx} (총 문서 수: {len(documents)})")


검색 결과:
🔹 문서 854: 개인용 자동차보험 특별약관
🔹 문서 5566: 이륜자동차보험 보통약관
🔹 문서 1058: 다이렉트개인용 자동차보험 보통약관
🔹 문서 27403: E10.31 당뇨병성 증식성 망막병증을 동반한 1형 당뇨병(H36.0*) E10.32 기타 및 상세불명의 망막병증을 동반한 1형 당뇨병(H36.0*) E10.33 당뇨병성 비증식성 망막병증을 동반한 2형 당뇨병(H36.0*) E11.31 당뇨병성 증식성 망막병증을 동반한 2형 당뇨병(H36.0*) E11.32 기타 및 상세불명의 망막병증을 동반한 2형 당뇨
🔹 문서 19512: 추간판 관련 경막외 신경차단술 - 치, 치수, 치은, 치근, 치조골의 처치 ※ 본 시술들은 수술의 정의에 해당하지 않는 시술의 예시로, 예시에 기재되어 있지 않다 하더라 도 수술의 정의에 해당하지 않는 경우 보상되지 않습니다. 101


# ERROR

In [9]:
# 1️⃣ 저장된 FAISS 인덱스 로드
index = faiss.read_index("./faiss_index_1.bin")

# 2️⃣ 문서 정보 로드
with open("./documents_1.pkl", "ab") as f:
    documents = pickle.load(f)

# 3️⃣ 검색 수행
query = " 보험금의 종류 및 한도에 대해 설명해줘?"
# query = "의무보험 미가입에 따른 불이익을 알려줘"
query_embedding = get_embeddings([query])[0]  # 쿼리 임베딩
query_embedding = np.array([query_embedding], dtype=np.float32)

D, I = index.search(query_embedding, k=5)  # 가장 유사한 5개 검색

# 4️⃣ 검색 결과 출력
for idx in I[0]:
    print(f"🔹 문서 {idx}: {documents[idx].page_content[:200]}")


UnsupportedOperation: read